# Automatic Ligand parameterization tutorial using BioExcel Building Blocks (biobb)
***
This tutorial aims to illustrate the process of **ligand parameterization** for a **small molecule**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **Ibuprofen** small compound (3-letter code IBP, Drugbank code [DB01050](https://www.drugbank.ca/drugs/DB01050)), a non-steroidal **anti-inflammatory drug** (NSAID) derived from propionic acid and it is considered the first of the propionics. 

**OpenBabel and ACPype** packages are used to **add hydrogens**, **energetically minimize the structure**, and 
**generate parameters** for the **GROMACS** package. With *Generalized Amber Force Field (GAFF)* forcefield and *AM1-BCC* charges.  
***
**Biobb modules** used:

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch data to be consumed by the rest of the Biobb building blocks.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to manipulate chemistry data.
 
**Auxiliar libraries** used:

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [jupyter_contrib_nbextensions](https://github.com/ipython-contrib/jupyter_contrib_nbextensions): Contains a collection of community-contributed unofficial extensions that add functionality to the Jupyter notebook. 
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.

### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_ligand_parameterization.git
cd biobb_wf_ligand_parameterization
conda env create -f conda_env/environment.yml
conda activate biobb_ligand_parameterization_tutorial
jupyter nbextension enable python-markdown/main
jupyter-notebook biobb_wf_ligand_parameterization/notebooks/biobb_ligand_parameterization_tutorial.ipynb
``` 

Please execute the following commands before launching the Jupyter Notebook if you experience some issues with widgets such as NGL View (3D molecular visualization):

```console
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
```

***
### Pipeline steps:
 1. [Input Parameters](#input)
 2. [Fetching Ligand Structure](#fetch)
 3. [Add Hydrogen Atoms](#addh)
 4. [Energetically Minimize Hydrogen Atoms](#min)
 5. [Generating Ligand Parameters](#acpype)
 6. [Output Files](#output)
 7. [Questions & Comments](#questions)
 
***
![](https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png)
***

<a id="input"></a>
***
## Input parameters
**Input parameters** needed:
 - **ligandCode**: 3-letter code of the ligand structure (e.g. IBP)
 - **mol_charge**: Molecule net charge (e.g. -1)
 - **pH**: Acidity or alkalinity for the small molecule. Hydrogen atoms will be added according to this pH. (e.g. 7.4)

In [1]:
import nglview
import ipywidgets
import os

ligandCode = 'KCX'
mol_charge = -1
pH = 7.4

<a id="fetch"></a>
***
## Fetching ligand structure
Downloading **ligand structure** in **PDB format** from the IRB PDB MIRROR database.<br>
Alternatively, a **PDB file** can be used as starting structure. <br>
***
**Building Blocks** used:

 - [Ligand](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.ligand) from **biobb_io.api.ligand**
***

In [2]:
# Ligand: Download ligand structure from MMB PDB mirror REST API (https://mmb.irbbarcelona.org/api/)
# Import module
from biobb_io.api.ligand import ligand

# Create prop dict and inputs/outputs
input_structure = ligandCode + '.pdb'

prop = {
    'ligand_code' : ligandCode
}

#Create and launch bb
ligand(output_pdb_path=input_structure,
        properties=prop)




2023-07-31 13:43:39,172 [MainThread  ] [INFO ]  Executing biobb_io.api.ligand Version: 4.0.0
2023-07-31 13:43:39,229 [MainThread  ] [INFO ]  Downloading kcx from: ftp://ftp.ebi.ac.uk/pub/databases/msd/pdbechem_v2/K/KCX/KCX_ideal.pdb
2023-07-31 13:43:39,231 [MainThread  ] [INFO ]  Writting pdb to: KCX.pdb


0

In [ ]:

# remove Hydrogen atoms to a small molecule
# Import module


### Visualizing 3D structure
Visualizing the downloaded/given **ligand PDB structure** using **NGL**:    

In [3]:
#Show small ligand structure
view = nglview.show_structure_file(input_structure)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','300px'])
view.camera='orthographic'
view

NGLWidget()

In [7]:
# remove carbamyl hydrogen

# Read in the file
with open(input_structure, 'r') as file :
  filedata = file.readlines()

# Remove the line containing HQ2
filedata = [line for line in filedata if "HXT" not in line]


deprot_structure = input_structure.replace(".pdb","_noHQ2.pdb")

# Write the file out again
with open(deprot_structure, 'w') as file:
  file.writelines(filedata)

#Show small ligand structure
view = nglview.show_structure_file(deprot_structure)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','300px'])
view.camera='orthographic'
view

NGLWidget()

<a id="addh"></a>
***
## Add Hydrogen Atoms
Adding **Hydrogen atoms** to the small molecule, according to the given pH.
***
**Building Blocks** used:
 - [BabelAddHydrogens](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_add_hydrogens) from **biobb_chemistry.babelm.babel_add_hydrogens** 
***

In [9]:
# Babel_add_hydrogens: add Hydrogen atoms to a small molecule
# Import module
from biobb_chemistry.babelm.babel_add_hydrogens import babel_add_hydrogens

# Create prop dict and inputs/outputs
output_babel_h = ligandCode + '.H.mol2' 

prop = {
#     'ph' : pH,
    'input_format' : 'pdb',
    'output_format' : 'mol2'
}
# break
# we do not need to add hydrogens
#Create and launch bb
babel_add_hydrogens(input_path=deprot_structure,
                  output_path=output_babel_h,
                  properties=prop)

2023-07-31 13:47:31,550 [MainThread  ] [INFO ]  Executing biobb_chemistry.babelm.babel_add_hydrogens Version: 4.0.0
2023-07-31 13:47:31,553 [MainThread  ] [INFO ]  Copy: /home/jovyan/biobb_wf_ligand_parameterization/notebooks/KCX_noHQ2.pdb to /home/jovyan/biobb_wf_ligand_parameterization/notebooks/7ea66a24-75e9-404d-b2a3-6c0ab3385a0c
2023-07-31 13:47:31,554 [MainThread  ] [INFO ]  Value  is not compatible as a coordinates value
2023-07-31 13:47:31,555 [MainThread  ] [INFO ]  obabel -ipdb /home/jovyan/biobb_wf_ligand_parameterization/notebooks/7ea66a24-75e9-404d-b2a3-6c0ab3385a0c/KCX_noHQ2.pdb -omol2 -O/home/jovyan/biobb_wf_ligand_parameterization/notebooks/7ea66a24-75e9-404d-b2a3-6c0ab3385a0c/KCX.H.mol2  -h 

2023-07-31 13:47:31,603 [MainThread  ] [INFO ]  Exit code 0

2023-07-31 13:47:31,605 [MainThread  ] [INFO ]  ==============================
*** Open Babel Warning  in parseConectRecord
  Problems reading a CONECT record.
  According to the PDB specification,
  Atoms with serial #1

0

### Visualizing 3D structure
Visualizing the **ligand PDB structure** with the newly added **hydrogen atoms** using **NGL**:    

In [10]:
#Show small ligand structure
view = nglview.show_structure_file(output_babel_h)
view.add_representation(repr_type='ball+stick', selection='all')
view.camera='orthographic'
view

NGLWidget()

<a id="min"></a>
***
## Energetically minimize Hydrogen Atoms
Energetically minimize newly added **Hydrogen atoms**.
***
**Building Blocks** used:
 - [BabelMinimize](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_minimize) from **biobb_chemistry.babelm.babel_minimize** 
***

In [11]:
# Babel_minimize: Structure energy minimization of a small molecule after being modified adding hydrogen atoms
# Import module
from biobb_chemistry.babelm.babel_minimize import babel_minimize

# Create prop dict and inputs/outputs
output_babel_min = ligandCode + '.H.min.pdb'                              
prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}


#Create and launch bb
babel_minimize(input_path=deprot_structure,
              output_path=output_babel_min,
              properties=prop)

2023-07-31 13:48:29,898 [MainThread  ] [INFO ]  Executing biobb_chemistry.babelm.babel_minimize Version: 4.0.0
2023-07-31 13:48:29,901 [MainThread  ] [INFO ]  Copy: /home/jovyan/biobb_wf_ligand_parameterization/notebooks/KCX_noHQ2.pdb to /home/jovyan/biobb_wf_ligand_parameterization/notebooks/eea4f504-5cce-4163-93aa-354f02520fb4
2023-07-31 13:48:29,902 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2023-07-31 13:48:29,904 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2023-07-31 13:48:29,905 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2023-07-31 13:48:29,906 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2023-07-31 13:48:29,907 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2023-07-31 13:48:29,909 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 10
2023-07-31 13:48:29,910 [MainThread  ] [INFO ]  obminimize -c 

2023-07-31 13:48:30,530 [MainThread  ] [INFO ]  Removed: ['/home/jovyan/biobb_wf_ligand_parameterization/notebooks/eea4f504-5cce-4163-93aa-354f02520fb4']


0

### Visualizing 3D structure
Visualizing the **ligand PDB structure** with the newly added **hydrogen atoms**, **energetically minimized**, using **NGL**:    

In [12]:
#Show small ligand structure
view = nglview.show_structure_file(output_babel_min)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','300px'])
view.camera='orthographic'
view

NGLWidget()

### Visualizing 3D structures
Visualizing all the structures generated so far:

 - Original **ligand PDB structure** (left)
 - **Ligand PDB structure** with **hydrogen atoms** (middle)
 - **Ligand PDB structure** with **hydrogen atoms energetically minimized** (right)  

In [13]:
#Show different structures generated (for comparison)
view1 = nglview.show_structure_file(input_structure)
view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['250px','300px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(output_babel_h)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['250px','300px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(output_babel_min)
view3.add_representation(repr_type='ball+stick')
view3._remote_call('setSize', target='Widget', args=['250px','300px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<a id="acpype"></a>
***
## Generating ligand parameters
**Building GROMACS topology** corresponding to the **ligand structure**.

**Force field** used in this tutorial step is **amberGAFF**: [General AMBER Force Field](http://ambermd.org/antechamber/gaff.html), designed for rational drug design.

***
**Building Blocks** used:
- [AcpypeParamsGMX](https://biobb-chemistry.readthedocs.io/en/latest/acpype.html#module-acpype.acpype_params_gmx) from **biobb_chemistry.acpype.acpype_params_gmx** 
***

In [14]:
# Acpype_params_gmx: Generation of topologies for GROMACS with ACPype
# Import module
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx

# Create prop dict and inputs/outputs
output_acpype_gro = ligandCode + 'params.gro'
output_acpype_itp = ligandCode + 'params.itp'
output_acpype_top = ligandCode + 'params.top'
output_acpype = ligandCode + 'params'
prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

#Create and launch bb
acpype_params_gmx(input_path=output_babel_min,
                output_path_gro=output_acpype_gro,
                output_path_itp=output_acpype_itp,
                output_path_top=output_acpype_top,
                properties=prop)

2023-07-31 13:48:47,457 [MainThread  ] [INFO ]  Executing biobb_chemistry.acpype.acpype_params_gmx Version: 4.0.0
2023-07-31 13:48:47,459 [MainThread  ] [INFO ]  Copy: /home/jovyan/biobb_wf_ligand_parameterization/notebooks/KCX.H.min.pdb to /home/jovyan/biobb_wf_ligand_parameterization/notebooks/a9ddec4d-1324-4a39-8841-b53ff13734c6
2023-07-31 13:48:47,461 [MainThread  ] [INFO ]  acpype -i /home/jovyan/biobb_wf_ligand_parameterization/notebooks/a9ddec4d-1324-4a39-8841-b53ff13734c6/KCX.H.min.pdb -b KCXparams.epektT -n -1

2023-07-31 13:49:05,281 [MainThread  ] [INFO ]  Exit code 0

2023-07-31 13:49:05,284 [MainThread  ] [INFO ]  ==========================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2022.6.6 (c) 2023 AWSdS |
==> ... charge set to -1
==> ... converting pdb input file to mol2 input file
==> * Babel OK *
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
==> * Tleap OK *
==> Remov

0

In [26]:
# !ls
files = "KCXparams.gro", "KCXparams.itp", "KCXparams.top"

for file in files:
#     print(files)
    print(file)
    with open(file, 'r') as f :
      filedata = f.readlines()
    
    for line in filedata:
        print(line)

('KCXparams.gro', 'KCXparams.itp', 'KCXparams.top')
KCXparams.gro
KCXparams_GMX.gro created by acpype (v: 2022.6.6) on Mon Jul 31 13:48:47 2023

 26

    1  KCX    N    1   0.302   0.183  -0.014

    1  KCX   CA    2   0.294   0.045   0.037

    1  KCX   CB    3   0.169  -0.027  -0.019

    1  KCX   CG    4   0.037   0.038   0.029

    1  KCX   CD    5  -0.086  -0.040  -0.024

    1  KCX   CE    6  -0.217   0.026   0.023

    1  KCX   NZ    7  -0.335  -0.046  -0.026

    1  KCX    C    8   0.420  -0.033   0.002

    1  KCX    O    9   0.462  -0.043  -0.111

    1  KCX   CX   10  -0.461  -0.008   0.000

    1  KCX  OXT   11   0.457  -0.113   0.098

    1  KCX  OQ1   12  -0.490   0.089   0.067

    1  KCX  OQ2   13  -0.550  -0.086  -0.053

    1  KCX    H   14   0.312   0.182  -0.116

    1  KCX  HN2   15   0.217   0.234   0.007

    1  KCX   HA   16   0.287   0.049   0.146

    1  KCX  HB2   17   0.173  -0.028  -0.128

    1  KCX  HB3   18   0.171  -0.132   0.014

    1  KCX  HG2   19  

### Visualizing 3D structure
Visualizing the generated **GROMACS** gro structure corresponding to the parameterized **ligand PDB structure** using **NGL**:    

In [22]:
#Show small ligand structure
view = nglview.show_structure_file(output_acpype_gro)
view.add_representation(repr_type='ball+stick', selection='all')
view._remote_call('setSize', target='Widget', args=['','300px'])
view.camera='orthographic'
view

NGLWidget()

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_acpype_gro}}: **Structure** of the parameterized ligand in gro (GROMACS) format.
 - {{output_acpype_top}}: **Topology** of the parameterized ligand, including a reference to the {{output_acpype_itp}}.
 - {{output_acpype_itp}}: **Include Topology File (itp)** of the parameterized ligand, including the parameters information: bonds, angles, dihedrals, etc.

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
